<a href="https://colab.research.google.com/github/aekanun2020/2025-NT-LLM-RAG/blob/main/SENT_v5_Hands_on_5_Answering_within_Context(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ติดตั้ง dependencies
!pip install llama-index -q
!pip install llama-index-embeddings-huggingface -q
!pip install llama-index-vector-stores-opensearch -q
!pip install requests -q
!pip install nest_asyncio -q
!pip install torch -q

# Import modules
import re
import torch
import json
import requests
import nest_asyncio
import time
from typing import List, Dict, Any
from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.vector_stores.opensearch import OpensearchVectorStore, OpensearchVectorClient
from llama_index.core.vector_stores.types import VectorStoreQueryMode
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# Apply nest_asyncio to avoid runtime errors
nest_asyncio.apply()

# กำหนดค่า API endpoints
OPENSEARCH_ENDPOINT = "http://34.41.37.53:9200"
OPENSEARCH_INDEX = "aekanun_doc_index"
TEXT_FIELD = "content"
EMBEDDING_FIELD = "embedding"
OLLAMA_API_ENDPOINT = "http://34.46.98.58:11434/api/generate"
LLM_MODEL = "qwen2.5:32b"

# ฟังก์ชันทำความสะอาดข้อความจาก ellipsis
def clean_ellipsis(text):
    cleaned_text = re.sub(r'\.{3,}', '', text)
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)
    return cleaned_text.strip()

# ฟังก์ชันค้นหาข้อมูลโดยใช้ LlamaIndex เหมือนใน hands-on 3
def search_api(query: str, top_k: int = 5) -> Dict[str, Any]:
    """
    ค้นหาข้อมูลใช้ LlamaIndex เหมือนใน hands-on 3
    :param query: คำค้นหา
    :param top_k: จำนวนผลลัพธ์สูงสุดที่ต้องการ
    :return: ผลลัพธ์การค้นหา
    """
    try:
        # สร้าง embedding model ทุกครั้งที่ใช้งาน
        print("กำลังสร้าง index ใหม่...")

        # Check if CUDA is available for GPU acceleration
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        # สร้าง embedding model
        embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-m3", device=device)
        print(f"สร้าง embedding model สำเร็จ ใช้อุปกรณ์: {device}")

        # ตรวจสอบขนาดของ embedding
        embeddings = embed_model.get_text_embedding("test")
        dim = len(embeddings)
        print(f"ขนาด embedding: {dim}")

        # สร้าง client และ vector store
        client = OpensearchVectorClient(
            endpoint=OPENSEARCH_ENDPOINT,
            index=OPENSEARCH_INDEX,
            dim=dim,
            embedding_field=EMBEDDING_FIELD,
            text_field=TEXT_FIELD,
            search_pipeline="hybrid-search-pipeline",
        )
        vector_store = OpensearchVectorStore(client)

        # สร้าง storage context และ index
        storage_context = StorageContext.from_defaults(vector_store=vector_store)
        index = VectorStoreIndex.from_vector_store(
            vector_store,
            storage_context=storage_context,
            embed_model=embed_model
        )
        print("สร้าง index ใหม่สำเร็จ")

        # สร้าง retriever แบบไม่ตัดทอนผลลัพธ์ เหมือนใน hands-on 3
        retriever = index.as_retriever(
            similarity_top_k=top_k,
            vector_store_query_mode=VectorStoreQueryMode.HYBRID
        )

        # ค้นหาข้อมูล
        results = retriever.retrieve(query)
        print(f"ค้นหาข้อมูลสำเร็จ พบผลลัพธ์ {len(results)} รายการ")

        # แปลงผลลัพธ์ให้อยู่ในรูปแบบที่เหมาะสม
        formatted_results = []
        total_tokens = 0

        for i, result in enumerate(results):
            # Get full content of the node without truncation
            try:
                full_text = result.node.get_content()
            except AttributeError:
                full_text = getattr(result, "text", str(result))

            # เตรียมข้อมูล metadata
            metadata = result.metadata if hasattr(result, "metadata") else {}

            # ประมาณจำนวน tokens (4 ตัวอักษรต่อ 1 token)
            tokens = len(full_text) // 4
            total_tokens += tokens

            # สร้างข้อมูลผลลัพธ์
            formatted_result = {
                "text": full_text,
                "file_path": metadata.get("file_path", "unknown.md"),
                "tokens": tokens,
                "section": metadata.get("section", "N/A"),
                "score": result.score if hasattr(result, "score") else None
            }

            formatted_results.append(formatted_result)

        return {
            "results": formatted_results,
            "total_tokens": total_tokens
        }
    except Exception as e:
        print(f"เกิดข้อผิดพลาดในการค้นหาข้อมูล: {str(e)}")
        import traceback
        traceback.print_exc()
        return {"results": [], "total_tokens": 0}

# ฟังก์ชันแสดงผลลัพธ์แบบสวยงาม
def display_search_results(results: List[dict]):
    """
    แสดงผลลัพธ์การค้นหาในรูปแบบที่อ่านง่าย
    :param results: ผลลัพธ์การค้นหาที่ได้จาก search_api
    """
    if not results:
        print("ไม่พบผลลัพธ์การค้นหา")
        return

    for i, result in enumerate(results):
        print(f"\n{'='*80}")
        print(f"ผลลัพธ์ที่ {i+1}")
        print(f"{'-'*80}")

        # แสดงข้อมูลพื้นฐาน
        print(f"File Path: {result.get('file_path', 'ไม่มีข้อมูล')}")
        print(f"Tokens: {result.get('tokens', 'ไม่ระบุ')}")
        print(f"Section: {result.get('section', 'N/A')}")

        # แสดงเนื้อหา
        print(f"{'-'*80}")
        print("เนื้อหา:")
        print(f"{result.get('text', 'ไม่มีเนื้อหา')}")

    print(f"\n{'='*80}")
    print(f"พบผลลัพธ์ทั้งหมด {len(results)} รายการ")

# ฟังก์ชันการสร้าง prompt สำหรับ LLM (ที่ปรับปรุงแล้ว)
def create_llm_prompt(query: str, search_results: List[dict]) -> str:
    """
    สร้าง prompt สำหรับส่งไปให้ Qwen2.5 ตอบคำถาม
    :param query: คำถามจากผู้ใช้
    :param search_results: ผลลัพธ์การค้นหา
    :return: prompt ที่จัดรูปแบบสำหรับ Qwen2.5
    """

    prompt_parts = []

    # ส่วนแนะนำและคำสั่ง
    prompt_parts.append("You are a question answering assistant. Answer the question as truthful and helpful as possible.")
    prompt_parts.append("When answering, please:")
    prompt_parts.append("1. Break down complex problems into manageable steps (Chain-of-Thought)")
    prompt_parts.append("2. Use logical reasoning and make valid inferences from the context")
    prompt_parts.append("3. Critically evaluate information quality and relevance")
    prompt_parts.append("4. Show calculations clearly when needed")
    prompt_parts.append("5. Verify your answer for accuracy and consistency before finalizing")
    prompt_parts.append("")

    # เพิ่ม context จากผลการค้นหา
    for i, result in enumerate(search_results):
        context_block = f"""BEGININPUT
BEGINCONTEXT
source: document_{i+1}
ENDCONTEXT
{result['text']}
ENDINPUT"""
        prompt_parts.append(context_block)

    # เพิ่มคำถาม
    prompt_parts.append(f"\nQuestion: {query}")
    prompt_parts.append("\nYour thinking process:")

    return "\n".join(prompt_parts)

# ฟังก์ชันสำหรับเรียกใช้ LLM
def query_llm(prompt: str) -> str:
    """
    ส่งคำถามไปยัง LLM API และรับคำตอบกลับมา
    :param prompt: prompt ที่ต้องการส่งไปยัง LLM
    :return: คำตอบจาก LLM
    """
    payload = {
        "model": LLM_MODEL,
        "stream": False,
        "prompt": prompt,
    }

    try:
        response = requests.post(OLLAMA_API_ENDPOINT, json=payload)
        if response.status_code == 200:
            llm_output = response.json()["response"]
            return clean_ellipsis(llm_output)
        else:
            print(f"เกิดข้อผิดพลาดในการเรียกใช้ LLM: HTTP {response.status_code}")
            if response.text:
                print(f"รายละเอียด: {response.text}")
            return f"เกิดข้อผิดพลาดในการเรียกใช้ LLM: HTTP {response.status_code}"
    except Exception as e:
        print(f"เกิดข้อผิดพลาดในการเรียกใช้ LLM: {str(e)}")
        return f"เกิดข้อผิดพลาดในการเรียกใช้ LLM: {str(e)}"

# ฟังก์ชันการค้นหาและตอบคำถามด้วย LLM
def search_and_answer(query: str, top_k: int = 5) -> Dict[str, Any]:
    """
    ค้นหาข้อมูลผ่าน API และใช้ LLM ตอบคำถาม
    :param query: คำถามจากผู้ใช้
    :param top_k: จำนวนผลลัพธ์สูงสุดที่ต้องการ
    :return: ข้อมูลผลลัพธ์การค้นหาและคำตอบจาก LLM
    """
    # 1. ค้นหาข้อมูลผ่าน API
    print(f"\n{'='*80}")
    print(f"กำลังค้นหาข้อมูลสำหรับคำถาม: {query}")
    search_response = search_api(query, top_k)

    search_results = search_response.get("results", [])
    total_tokens = search_response.get("total_tokens", 0)

    # 2. แสดงผลลัพธ์การค้นหา
    if search_results:
        display_search_results(search_results)
    else:
        print("ไม่พบผลลัพธ์การค้นหา หรือเกิดข้อผิดพลาดในการค้นหา")
        return {
            "query": query,
            "search_results": [],
            "llm_prompt": "",
            "llm_answer": "ไม่พบข้อมูลที่เกี่ยวข้อง",
            "total_tokens": 0
        }

    # 3. สร้าง prompt สำหรับ LLM
    llm_prompt = create_llm_prompt(query, search_results)

    # 4. เรียกใช้ LLM
    print(f"\n{'='*80}")
    print("Prompt ที่ส่งไปยัง LLM:")
    print(f"{'-'*80}")
    print(llm_prompt)
    print(f"{'-'*80}")
    print("\nกำลังสร้างคำตอบจาก LLM...")
    llm_answer = query_llm(llm_prompt)

    # 5. แสดงคำตอบจาก LLM
    print(f"\n{'='*80}")
    print("คำตอบจาก LLM:")
    print(f"{'-'*80}")
    print(llm_answer)
    print(f"\n{'='*80}")

    # 6. แสดงจำนวน token ทั้งหมด
    print(f"จำนวน tokens ทั้งหมด: {total_tokens}")

    # 7. ส่งคืนข้อมูลทั้งหมด
    return {
        "query": query,
        "search_results": search_results,
        "llm_prompt": llm_prompt,
        "llm_answer": llm_answer,
        "total_tokens": total_tokens
    }

# คำถามที่กำหนดไว้ล่วงหน้า
predefined_questions = [
        "โรคหัดและโรคหัดเยอรมันแตกต่างกันอย่างไร?",  # คำถามเปรียบเทียบ
        "อธิบายสาเหตุของโรคหัดเยอรมันและการป้องกัน",  # คำถามหลายประเด็น
        "ทำไมโรคหัดเยอรมันจึงมีอันตรายกับหญิงตั้งครรภ์?",  # คำถามวิเคราะห์เชิงลึก
        "ถ้าคนที่ฉีดวัคซีนป้องกันโรคหัดเยอรมันแล้ว จะมีโอกาสติดเชื้อหรือไม่?",  # คำถามสมมติเหตุการณ์
        "โรคหัดเยอรมันมีผลกระทบอย่างไรต่อระบบสาธารณสุขและเศรษฐกิจของประเทศ?",  # คำถามข้ามสาขา
        "การรักษาโรคหัดเยอรมันที่ดีที่สุดคืออะไร?"  # คำถามกำกวม
    ]

# ดำเนินการหลัก
print("กำลังเริ่มต้นระบบค้นหาข้อมูลและตอบคำถามสุขภาพ...")

# ตรวจสอบการเชื่อมต่อกับ LLM API
try:
    test_payload = {
        "model": LLM_MODEL,
        "stream": False,
        "prompt": "สวัสดี ทดสอบการเชื่อมต่อ"
    }
    response = requests.post(OLLAMA_API_ENDPOINT, json=test_payload)
    if response.status_code == 200:
        print(f"เชื่อมต่อกับ LLM API ({OLLAMA_API_ENDPOINT}) สำเร็จ")
    else:
        print(f"เชื่อมต่อกับ LLM API ({OLLAMA_API_ENDPOINT}) ล้มเหลว: HTTP {response.status_code}")
except Exception as e:
    print(f"เกิดข้อผิดพลาดในการเชื่อมต่อกับ LLM API: {str(e)}")
    print("คุณอาจจะต้องเริ่มต้น Ollama service ก่อนโดยใช้คำสั่ง 'ollama serve' และตรวจสอบให้แน่ใจว่าได้ดาวน์โหลดโมเดล LLM แล้ว")

# ทดสอบการค้นหาและตอบคำถามด้วยคำถามที่กำหนดไว้ล่วงหน้า
print("\nเริ่มการทดสอบค้นหาและตอบคำถามด้วยคำถามที่กำหนดไว้ล่วงหน้า:")
print(f"{'-'*80}")

for i, question in enumerate(predefined_questions):
    print(f"\nคำถามที่ {i+1}: {question}")
    search_and_answer(question)

    # ถามผู้ใช้ว่าต้องการดูคำถามถัดไปหรือไม่
    if i < len(predefined_questions) - 1:
        user_input = input("\nกด Enter เพื่อดูผลลัพธ์ของคำถามถัดไป หรือพิมพ์ 'q' เพื่อออก: ")
        if user_input.lower() == 'q':
            break

print("\nเสร็จสิ้นการทดสอบคำถามที่กำหนดไว้ล่วงหน้า!")

# อนุญาตให้ผู้ใช้ป้อนคำถามเพิ่มเติม
print("\nคุณสามารถป้อนคำถามเพิ่มเติมได้:")
while True:
    user_question = input("\nป้อนคำถามของคุณ (หรือพิมพ์ 'exit' เพื่อออก): ")
    if user_question.lower() == 'exit':
        break

    search_and_answer(user_question)

print("\nขอบคุณที่ใช้ระบบค้นหาข้อมูลและตอบคำถามสุขภาพ!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.3/251.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 56.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/15.8k [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

Streaming output truncated to the last 5000 lines.
source: document_4
ENDCONTEXT
## สมุนไพรรักษาหัดเยอรมัน (ตามองค์ความรู้ดั้งเดิม) มีอยู่หลายชนิด เช่น

- ปลาไหลเผือก (Eurycoma longifolia Jack) ใช้เปลือกลำต้นนำมาต้มเอาน้ำกินเป็นยาแก้ไข้เหือดหัด

- ผักหนาม (Lasia spinosa (L.) Thwaites) ใช้ลำต้นนำมาต้มเอาน้ำอาบแก้อาการคันเนื่องจากพิษหัด เหือด ไข้ออกผื่น สุกใส ดำแดง ทำให้ผื่นหายเร็ว

- เสลดพังพอนตัวเมีย (Clinacanthus nutans (Burm.f.) Lindau) ใช้ใบสดประมาณ 7 กำมือ นำมาต้มกับน้ำ 8 แก้ว ต้มให้เดือด 30 นาที เทยาออกและผึ่งให้เย็น แล้วนำใบสดมาอีก 7 กำมือ ตำผสมกับน้ำ 8 แก้ว แล้วเอาน้ำยาทั้งสองมาผสมกัน ใช้ทั้งกินและชโลมทา (ยาชโลมให้ใส่พิมเสนลงไปเล็กน้อย) เด็กที่เป็นหัด เหือด ให้กินวันละ 3 ครั้ง ครั้งละครึ่งแก้ว

- หญ้าแพรก (Cynodon dactylon (L.) Pers.) ทั้งต้นใช้ต้มกับน้ำกินเป็นยาแก้ไข้พิษ ไข้กาฬ ไข้หัว รวมถึงเหือดหัด

- สมุนไพรอื่น ๆ เช่น หัวของต้นคล้า (Schumannianthus dichotomus (Roxb.) Gagnep.), รากผักชี (Coriandrum sativum L.), ใบมะยม (Phyllanthus acidus (L.) Skeels), รากหญ้าขัดใบมน (Sida rho